<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [12]:
import numpy as np
import pandas as pd
import gensim
import os
import re
import zipfile

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

In [35]:
data_path = os.path.join(os.getcwd(), '../module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip')

archive = zipfile.ZipFile(data_path)
files = archive.namelist()

archive.extract(files[0])
df = pd.read_csv(files[0])
os.remove(files[0])

In [42]:
df.shape

(28332, 25)

In [37]:
def tokenize(text):
    return [token for token in simple_preprocess(text, deacc=True) if token not in STOPWORDS]

In [38]:
df['tokens'] = df['reviews.text'].apply(lambda x: tokenize(x))
df['tokens'].head()

0    [order, item, bad, quality, missing, backup, s...
1               [bulk, expensive, way, products, like]
2                             [duracell, price, happy]
3              [work, brand, batteries, better, price]
4             [batteries, long, lasting, price, great]
Name: tokens, dtype: object

In [47]:
# id2word = df['tokens'].apply(lambda x: corpora.Dictionary(x))
id2word = corpora.Dictionary(df['tokens'])

In [48]:
import sys

sys.getsizeof(id2word)

56

In [49]:
len(id2word.keys())

9620

In [55]:
id2word.filter_extremes(no_below=5, no_above=0.95)

In [56]:
len(id2word.keys())

3581

In [57]:
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [58]:
corpus[0][:10]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

In [59]:
lda = LdaMulticore(corpus=corpus,
                  id2word=id2word,
                  random_state=42,
                  num_topics=10,
                  passes=10,
                  workers=4)

In [60]:
lda.print_topics()

[(0,
  '0.093*"good" + 0.041*"best" + 0.040*"buy" + 0.040*"product" + 0.033*"amazon" + 0.027*"quality" + 0.027*"price" + 0.016*"excellent" + 0.014*"like" + 0.014*"value"'),
 (1,
  '0.054*"kindle" + 0.036*"love" + 0.035*"easy" + 0.022*"size" + 0.022*"use" + 0.022*"screen" + 0.020*"great" + 0.016*"reading" + 0.016*"tablet" + 0.016*"nice"'),
 (2,
  '0.048*"tablet" + 0.035*"games" + 0.032*"kids" + 0.029*"old" + 0.026*"year" + 0.018*"time" + 0.016*"loves" + 0.015*"apps" + 0.015*"love" + 0.015*"bought"'),
 (3,
  '0.047*"batteries" + 0.021*"amazon" + 0.015*"use" + 0.012*"box" + 0.010*"battery" + 0.009*"like" + 0.009*"work" + 0.009*"ve" + 0.007*"ok" + 0.007*"time"'),
 (4,
  '0.045*"kindle" + 0.017*"new" + 0.017*"charge" + 0.014*"like" + 0.013*"better" + 0.011*"read" + 0.010*"buy" + 0.010*"screen" + 0.009*"battery" + 0.009*"time"'),
 (5,
  '0.119*"great" + 0.080*"tablet" + 0.043*"use" + 0.034*"easy" + 0.029*"price" + 0.028*"works" + 0.022*"product" + 0.014*"recommend" + 0.014*"good" + 0.012*"lo

In [62]:
# Strip out everything except the words
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [63]:
topics = [' '.join(t[0:5]) for t in words]

In [64]:
for id, t in enumerate(topics):
    print(f"----- topic {id} -----")
    print(t, end="\n\n")

----- topic 0 -----
good best buy product amazon

----- topic 1 -----
kindle love easy size use

----- topic 2 -----
tablet games kids old year

----- topic 3 -----
batteries amazon use box battery

----- topic 4 -----
kindle new charge like better

----- topic 5 -----
great tablet use easy price

----- topic 6 -----
tablet amazon apps screen books

----- topic 7 -----
batteries use tablet toys remote

----- topic 8 -----
bought gift kids loves love

----- topic 9 -----
batteries great price good long



In [65]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [66]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\Users\theta\Anaconda3\envs\u4_s1_nlp\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.057508  0.054630       1        1  14.650017
9      0.247413 -0.132351       2        1  14.388789
2     -0.197674  0.004861       3        1  12.323174
3      0.205770  0.060958       4        1  11.068825
5     -0.123922 -0.153962       5        1   9.944622
4      0.039922  0.236060       6        1   8.629407
1     -0.164454  0.075794       7        1   8.615110
0      0.147692 -0.144015       8        1   7.338686
8     -0.201873 -0.097038       9        1   6.608390
7      0.104633  0.095064      10        1   6.432987, topic_info=     Category         Freq       Term        Total  loglift  logprob
18    Default  6922.000000  batteries  6922.000000  30.0000  30.0000
21    Default  8776.000000      great  8776.000000  29.0000  29.0000
34    Default  5660.000000       good  5660.000000  28.0000  28.0000
2432  Default  6858.000000     tablet  6858.000000  27.0000  27.0000
2075  Default  3071.000000     kindle  3071.000000  26.0000  26.0000
17    Default  5027.000000      price  5027.000000  25.0000  25.0000
192   Default  2532.000000       kids  2532.000000  24.0000  24.0000
23    Default  2410.000000       long  2410.000000  23.0000  23.0000
1857  Default  2227.000000      loves  2227.000000  22.0000  22.0000
885   Default  1104.000000       gift  1104.000000  21.0000  21.0000
755   Default  2900.000000       easy  2900.000000  20.0000  20.0000
165   Default  1910.000000    product  1910.000000  19.0000  19.0000
28    Default  3196.000000     bought  3196.000000  18.0000  18.0000
652   Default  1721.000000      games  1721.000000  17.0000  17.0000
90    Default  3462.000000       love  3462.000000  16.0000  16.0000
521   Default  2196.000000        old  2196.000000  15.0000  15.0000
31    Default   920.000000  christmas   920.000000  14.0000  14.0000
105   Default  1836.000000       year  1836.000000  13.0000  13.0000
222   Default  1468.000000       best  1468.000000  12.0000  12.0000
29    Default  2259.000000        buy  2259.000000  11.0000  11.0000
1548  Default  1853.000000       apps  1853.000000  10.0000  10.0000
2     Default  2240.000000    battery  2240.000000   9.0000   9.0000
94    Default  4420.000000        use  4420.000000   8.0000   8.0000
7     Default  1302.000000    quality  1302.000000   7.0000   7.0000
1679  Default  1638.000000     screen  1638.000000   6.0000   6.0000
96    Default  1358.000000      works  1358.000000   5.0000   5.0000
20    Default  1176.000000      brand  1176.000000   4.0000   4.0000
9     Default  2221.000000       work  2221.000000   3.0000   3.0000
129   Default   958.000000       size   958.000000   2.0000   2.0000
74    Default   805.000000     brands   805.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
136   Topic10   149.990936     lasted   285.376831   2.1005  -4.8820
137   Topic10   124.833107      month   233.675262   2.1168  -5.0656
589   Topic10    43.315643   operated    61.483849   2.3935  -6.1241
679   Topic10    26.316206   specific    31.436844   2.5659  -6.6224
499   Topic10   167.716492   controls   413.865051   1.8405  -4.7703
1866  Topic10    87.002319    toddler   169.742981   2.0754  -5.4267
184   Topic10    93.668602      weeks   200.622467   1.9821  -5.3529
405   Topic10   141.870270      going   427.425964   1.6409  -4.9377
380   Topic10    52.546764        toy    88.723480   2.2199  -5.9309
18    Topic10   775.731506  batteries  6922.271484   0.5550  -3.2388
511   Topic10   104.802696        won   284.613770   1.7447  -5.2405
218   Topic10   119.654625      hours   357.733398   1.6486  -5.1080
678   Topic10   187.762573        set   854.031128   1.2289  -4.6574
98    Topic10   108.159966       days   328.727081   1.6321  -5.2090
106   Topic10   189.821198       need   951.183105   1.1321  -4.6465
94    Topic10   411.624146        use  4420.575195   0.3698  -3.87

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling